In [34]:
!pip install --user git+https://github.com/GerbenBeintema/deepSI@master

  Cloning https://github.com/GerbenBeintema/deepSI (to revision master) to c:\users\klaem\appdata\local\temp\pip-req-build-1sj17_38
  Resolved https://github.com/GerbenBeintema/deepSI to commit 28c96c174fa2e1c83aeb26091d67785d468a4bee
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/GerbenBeintema/deepSI 'C:\Users\klaem\AppData\Local\Temp\pip-req-build-1sj17_38'

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\klaem\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [35]:
import sys
#sys.path.insert(0, '/gpfs/scratch1/nodespecific/gcn3/klaemers.6208466/pip-req-build-cdv8bmd8')
import deepSI
import numpy as np
import cv2
from deepSI import System_data
from matplotlib import pyplot as plt

In [36]:
# Load the npz file
filen = 'combined_data_dome1.npz'
data = np.load(filen, allow_pickle=True)

# Assuming you have arrays 'frames' and 'forces' within your npz file
frames = data['frames']
forces = data['forces']

# Determine the size of each set
total_size = len(frames)
train_size = int(total_size * 0.5)  # 50% of the data for training
val_size = int(total_size * 0.20)  # 20% of the data for validation
test_size = total_size - train_size - val_size  # Remaining 30% for testing

# Function to resize and reshape frames
def resize_and_reshape_frames(frames, batch_size, new_height, new_width):
    num_frames = frames.shape[0]
    resized_and_reshaped_frames = np.zeros((num_frames, frames.shape[3], new_height, new_width), dtype=np.uint8)
    for start in range(0, num_frames, batch_size):
        end = start + batch_size
        batch_frames = frames[start:end]
        for i in range(batch_frames.shape[0]):
            resized_frame = cv2.resize(batch_frames[i], (new_width, new_height), interpolation=cv2.INTER_LINEAR)
            resized_and_reshaped_frames[start + i] = resized_frame.transpose(2, 0, 1)
    return resized_and_reshaped_frames

# Resize and reshape parameters
new_height = frames.shape[1] // 2
new_width = frames.shape[2] // 2
batch_size = 30

# Resize and reshape all frames
frames_resized_reshaped = resize_and_reshape_frames(frames, batch_size, new_height, new_width)

In [37]:
# Split the data
frames_train = frames_resized_reshaped[:train_size]
frames_val = frames_resized_reshaped[train_size:train_size + val_size]
frames_test = frames_resized_reshaped[train_size + val_size:]

forces_train = forces[:train_size, 3]
forces_val = forces[train_size:train_size + val_size, 3]
forces_test = forces[train_size + val_size:, 3]

# Initialize the SS_encoder_CNN_video system
sys_vbss = deepSI.fit_systems.SS_encoder_CNN_video(na=20, nb=20)
n_channels, height, width = frames_train.shape[1], frames_train.shape[2], frames_train.shape[3]
sys_vbss.init_nets(nu=1, ny=(n_channels, height, width))

# Create System_data instances with resized and reshaped frames
system_data = System_data(u=forces_train, y=frames_train)
system_data_val = System_data(u=forces_val, y=frames_val)
system_data_test = System_data(u=forces_test, y=frames_test)
sys_vbss.n_step_error()

In [38]:
# Customize normalization for training, validation, and test sets
#sys_vbss.use_norm = False  # Disable automatic normalization

# Normalize the frames by computing mean and standard deviation over samples, height, and width
#sys_vbss.norm.y0 = np.mean(system_data.y, axis=(0, 2, 3))[:, None, None]
#sys_vbss.norm.ystd = np.std(system_data.y, axis=(0, 2, 3))[:, None, None]

# Normalize forces by computing mean and standard deviation over samples
#sys_vbss.norm.u0 = np.mean(system_data.u, axis=0)
#sys_vbss.norm.ustd = np.std(system_data.u, axis=0)

In [52]:
# Fit the model
#sys.fit(system_data, val_sys_data=system_data_val, cuda=True, batch_size=64, loss_kwargs={'online_construct': True})
name = "model"
sys_vbss = deepSI.load_system_data(name)


AttributeError: 'bytes' object has no attribute 'shape'

In [ ]:
# Save the model
#sys.save_system('model2_best')

In [ ]:
test_p = sys_vbss.apply_experiment(system_data_test)

In [ ]:
from plot_scripts import strip_plotter

In [ ]:
to_img = lambda image: np.transpose(np.clip(np.round(image), 0, 255).astype(np.uint8), (1, 2, 0))
strip_plotter(system_data_test, test_p, sys_vbss.norm, to_img=to_img, filename='figures/strip-UD.jpg')

In [ ]:
from plot_scripts import make_video_as_avi

In [ ]:
k = test_p.cheat_n
make_video_as_avi(system_data_test[k:k+30*60],test_p[k:k+30*60],sys_vbss.norm, to_img=to_img, target_fps=30, filename='movies/UD-test-short.mp4')